In [4]:
import numpy as np
import matplotlib.pyplot as plt
import glob,sys, dill

sys.path.append('../src')
from lib.utils import sort_fn

In [20]:
l2_x_data = sorted(
    glob.glob(f"../data/SPCAM5/inputs_*"), key=sort_fn)
l2_y_data = sorted(
    glob.glob(f"../data/SPCAM5/outputs_*"), key=sort_fn)
l1_x_data = sorted(
    glob.glob(f"../data/CAM5/inputs_*"), key=sort_fn)
l1_y_data = sorted(
    glob.glob(f"../data/CAM5/outputs_*"), key=sort_fn)

split_n = int(365*0.8)
l2_x_train = l2_x_data[:split_n]
l2_y_train = l2_y_data[:split_n]
l2_x_valid = l2_x_data[split_n:]
l2_y_valid = l2_y_data[split_n:]
l1_x_train = l1_x_data[:split_n]
l1_y_train = l1_y_data[:split_n]
l1_x_valid = l1_x_data[split_n:]
l1_y_valid = l1_y_data[split_n:]

l2_x_scaler_minmax = dill.load(
    open(f"../scalers/x_SPCAM5_minmax_scaler.dill", 'rb'))
l2_y_scaler_minmax = dill.load(
    open(f"../scalers/y_SPCAM5_minmax_scaler.dill", 'rb'))
l1_x_scaler_minmax = dill.load(
    open(f"../scalers/x_CAM5_minmax_scaler.dill", 'rb'))
l1_y_scaler_minmax = dill.load(
    open(f"../scalers/y_CAM5_minmax_scaler.dill", 'rb'))

# Change to first 26 variables
l2_y_scaler_minmax.min = l2_y_scaler_minmax.min[:26]
l2_y_scaler_minmax.max = l2_y_scaler_minmax.max[:26]
l1_y_scaler_minmax.min = l1_y_scaler_minmax.min[:26]
l1_y_scaler_minmax.max = l1_y_scaler_minmax.max[:26]


In [26]:
for l1_x_, l1_y_, l2_x_, l2_y_ in zip(l1_x_data, l1_x_data, l2_x_data, l2_y_data):
    l1_x = np.load(l1_x_)
    l1_y = np.load(l1_y_)
    l2_x = np.load(l2_x_)
    l2_y = np.load(l2_y_)
    if l1_x.max() > 1e7:
        print('here', l1_x)
    
    l1_x_t = l1_x_scaler_minmax.inverse_transform(l1_x)
    l1_y_t = l1_y_scaler_minmax.inverse_transform(l1_y)
    l2_x_t = l2_x_scaler_minmax.inverse_transform(l2_x)
    l2_y_t = l2_y_scaler_minmax.inverse_transform(l2_y)

    print(l1_x_, end='\r')
    if l1_x_t.max() > 1e7 or l2_x_t.max() > 1e7:
        print('here 2', l1_x)
        print(l1_x.min(), l1_x.max(), l1_x_scaler_minmax.min.min(), l1_x_scaler_minmax.max.max())
    # break